<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/OpenAI_Whisper_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAIのWhisper音声認識モデルをデモするWebアプリケーション

このColabノートブックは、[OpenAIのフリーWhisper音声認識モデル](https://openai.com/blog/whisper/)に音声ファイルを録音またはアップロードするためのものです。これは[@amrrsによるオリジナルのノートブック](https://github.com/amrrs/openai-whisper-webapp)をベースに、[Pete Warden](https://twitter.com/petewarden)によるドキュメントとテストファイルを追加したものです。

使用するには、Colabメニューから`Runtime->Run All`を選択してください。このノートブックをGitHubで見ている場合は、まず[このリンク](https://colab.research.google.com/github/petewarden/openai-whisper-webapp/blob/main/OpenAI_Whisper_ASR_Demo.ipynb)をたどってColabで開いてください。1分ほどすると、ページの下に`Record from microphone`リンクのボタンが表示されるはずです。これをクリックすると、マイクへのアクセス許可を求められるので、最大30秒間話してください。録音が終わったら、「録音停止」を押してください。録音ボタンの右側のボックスに、最初の30秒間のスピーチ原稿が表示されます。さらに書き起こしたい場合は、左のボックスの「クリア」をクリックし、最初からやり直してください。

このページの左側にあるフォルダアイコンを使って、自分の音声サンプルをアップロードすることもできます。ファイルをドラッグしてアップロードできるファイルシステムにアクセスできます。以下のいくつかのセルで、テープ起こしの実行例を見ることができます。

## Whisper のインストール

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install torchinfo

## モデルのロード

In [ ]:
import whisper
model = whisper.load_model("medium") # tiny, base, samll, medium, large, turbo

## モデルの中身
torchinfo の summary 関数で中身を見られます。

In [ ]:
from torchinfo import summary
summary(model)

## GPUの確認

以下のように`device(type='cuda', index=0)`が出力されるはずです。もし表示されない場合、CPUのみのColabインスタンスを使用している可能性があり、動作が遅くなります。Runtime->Change Runtime Type`で修正してください。

In [ ]:
model.device

## テスト用音声ファイルのダウンロード

このリポジトリには、書き起こし機能を実行するためにあらかじめ録音された MP3 がいくつかあります。下に表示されているオーディオウィジェットで聴くことができます。

In [ ]:
!git clone https://github.com/petewarden/openai-whisper-webapp

In [ ]:
from IPython.display import Audio
Audio("/content/openai-whisper-webapp/two_cities.mp3")

In [ ]:
from IPython.display import Audio
Audio("/content/openai-whisper-webapp/daisy_HAL_9000.mp3")

## Transcribe 関数の定義

音声ファイルのパスを入力として受け取り、認識されたテキストを返します。

In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text


## 録音済みの音声でテストする

ライブ音声を録音するためのUIを立ち上げる前に、ダウンロードしたMP3をいくつか使って `transcribe()` 関数を実行してみましょう。クリアな音声の例として録音した `mary.mp3` からは `Mary had a little lamb, its fleece was white as snow, and everywhere that Mary went, the lamb was sure to go.` という認識結果が出力されます。2つ目のファイルは非常に歪んだ音声で、聞き取ることがかなり難しいですが、モデルは`Daisy, Daisy, give me your answer too. I'm half crazy all for the love of you.`と正しく認識しています。

注：このノートのデフォルトの長さである30秒でトランスクリプトが切れていることに気づくでしょう（設定の変更は可能）。

In [ ]:
easy_text = transcribe("/content/openai-whisper-webapp/mary.mp3")
print(easy_text)

hard_text = transcribe("/content/openai-whisper-webapp/daisy_HAL_9000.mp3")
print(hard_text)

## 長時間対応
transcribe 関数は30秒までしか使えないので、長時間に対応した関数 transcribe_long_audio を作ります。

In [ ]:
import whisper
import librosa
import numpy as np

def transcribe_long_audio(audio_path, chunk_duration=60):
  """
  長い音声ファイルを指定された長さのチャンクに分割して書き起こす。

  Args:
    audio_path: 音声ファイルのパス
    chunk_duration: チャンクの長さ（秒）

  Returns:
    書き起こし結果の文字列
  """

  #model = whisper.load_model("medium")
  audio, sr = librosa.load(audio_path, sr=whisper.audio.SAMPLE_RATE)

  chunk_length = chunk_duration * whisper.audio.SAMPLE_RATE
  num_chunks = int(np.ceil(len(audio) / chunk_length))

  transcription_text = ""

  for i in range(num_chunks):
    start = i * chunk_length
    end = min((i + 1) * chunk_length, len(audio))
    chunk = audio[start:end]

    # チャンクの長さが30秒未満の場合は、whisper.pad_or_trim でパディングする
    if len(chunk) < 30 * whisper.audio.SAMPLE_RATE:
      chunk = whisper.pad_or_trim(chunk)
    else:
      # 30秒より長いチャンクの場合は、さらに30秒ずつに分割する
      sub_chunks = [chunk[j*30*whisper.audio.SAMPLE_RATE:min((j+1)*30*whisper.audio.SAMPLE_RATE, len(chunk))] for j in range(int(np.ceil(len(chunk)/(30*whisper.audio.SAMPLE_RATE))))]
      for sub_chunk in sub_chunks:
        sub_chunk = whisper.pad_or_trim(sub_chunk)
        result = model.transcribe(sub_chunk)
        transcription_text += result["text"]
      continue

    result = model.transcribe(chunk)
    transcription_text += result["text"]

  return transcription_text

### 長い音声ファイルの認識

In [ ]:
transcription = transcribe_long_audio("/content/openai-whisper-webapp/two_cities.mp3", chunk_duration=60)
transcription

### 画面の端までいったら改行させる設定

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
transcription

## 多言語対応
Whisper は、英語以外の言語も自動的に認識します。

## ライブラリのインストール

In [ ]:
!pip install yt-dlp moviepy
!pip install gTTS

## ライブラリのインポート

In [ ]:
from yt_dlp import YoutubeDL
from moviepy.editor import VideoFileClip, AudioFileClip
import IPython

## 音声サンプルの入手

In [ ]:
video_url = 'https://youtu.be/G807ErB7ytY?si=KHTqIkjNVrf0Q1Ts' #@param {type:"string"}

## ダウンロード

In [ ]:
download_resolution = 360
input_clip_path = 'input_clip.mp4'
input_audio_path = 'input_clip.mp3'

# 動画ダウンロード --> input_clip.mp4
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': input_clip_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 音声だけ取り出し --> input_clip.mp3
!yes|ffmpeg -i input_clip.mp4 input_clip.mp3

## 確認（オーディオ）

In [ ]:
IPython.display.Audio('input_clip.mp3', autoplay=True)

## 時間調整
長いので、冒頭2分だけを切り出す。

In [ ]:
!yes|ffmpeg -i input_clip.mp3 -t 120 input_clip2.mp3

## 認識させてみる
30秒を超えているので、long_audio 関数で行う。

In [ ]:
transcription = transcribe_long_audio("input_clip2.mp3", chunk_duration=60)
transcription

## おまけ
機械に喋らせてみる。

In [ ]:
# 音声発話のためのライブラリ
from gtts import gTTS
gTTS(text = transcription, lang = "ja", slow = False).save('output.wav')
IPython.display.Audio('output.wav', autoplay=True)